In [216]:
import itertools as it

import numpy as np
import pandas as pd
from opre_tools import negated_bounded_span, print_full
from ortools.sat.python import cp_model

MAX_WKS = 52

In [237]:
prefs = pd.read_excel("input.xlsx",sheet_name="preferences",index_col="resident_name")
rotations = pd.read_excel("input.xlsx",sheet_name="rotations",index_col='rotation_name')
residents = pd.read_excel("input.xlsx",sheet_name="residents",index_col='resident_name')
weeks = pd.read_excel("input.xlsx",sheet_name="weeks",index_col='week_n')
categories = pd.read_excel("input.xlsx",sheet_name="categories",index_col='category')

In [238]:
prefs.head()

,CCNW H/O,STHC Clinic,Consults,DH ICU,PMP,Prov Pulm,Prov Rheum,Summit CC,Multicare Pulm,CICU
resident_name,,,,,,,,,,
Dewald,1.0,1.0,2.0,3.0,1.0,3.0,2.0,2.0,3.0,3.0
Fisher,2.0,2.0,1.0,3.0,2.0,2.0,1.0,2.0,2.0,3.0
Kirby,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,2.0
Peters,2.0,2.0,3.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0
Ruger,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [239]:
rotations.head()

,min_wks,min_contig_wks,max_wks,resident_capacity,required_level,category
rotation_name,,,,,,
PMP,0,2,4,1,Any,Primary Care
HS Senior,4,2,8,2,Senior,HS
HS Intern,8,4,8,4,Intern,HS
SHMC ICU Senior,4,4,4,1,Senior,ICU
Prov Rheum,0,2,4,1,Senior,Rheum


In [240]:
residents.head(30)

,resident_year
resident_name,
Dewald,3
Fisher,3
Kirby,3
Peters,3
Ruger,3
Ken Scott,3
Solaiman,3
Straus,3
Warr,3


In [241]:
weeks.head()

,week_start
week_n,
0,2025-07-01
1,2025-07-08
2,2025-07-15
3,2025-07-22
4,2025-07-29


In [242]:
# sig_categories = categories.query("(category_min_wks > 1) and (category_max_wks < @MAX_WKS)")
# sig_categories

In [243]:
# rotations_s_t_categories = pd.merge(sig_categories,rotations,left_index=True,right_on="category" )
# rotations_s_t_categories

In [244]:
rotations[(rotations.min_wks > 0) & (rotations.required_level == "Senior")]

,min_wks,min_contig_wks,max_wks,resident_capacity,required_level,category
rotation_name,,,,,,
HS Senior,4,2,8,2,Senior,HS
SHMC ICU Senior,4,4,4,1,Senior,ICU
STHC Amb,4,1,12,2,Senior,Primary Care
Nights Senior,4,4,4,1,Senior,Nights
Consults,4,1,4,2,Senior,Consults
Purple Senior,4,1,4,1,Senior,Purple


In [245]:
for r, w in it.product(rotations.index, weeks.index):
    print(r, rotations.resident_capacity[r])
    break

PMP 1


In [246]:
d.loc[pd.IndexSlice[:, r, w]]

Amanda Scott          bool_at_[('Amanda Scott', 'PMP', 0)]
Benito                      bool_at_[('Benito', 'PMP', 0)]
Cronrath                  bool_at_[('Cronrath', 'PMP', 0)]
Dewald                      bool_at_[('Dewald', 'PMP', 0)]
Eymann                      bool_at_[('Eymann', 'PMP', 0)]
Fisher                      bool_at_[('Fisher', 'PMP', 0)]
Gonzalez-Yetter    bool_at_[('Gonzalez-Yetter', 'PMP', 0)]
Kahn                          bool_at_[('Kahn', 'PMP', 0)]
Katsilometes          bool_at_[('Katsilometes', 'PMP', 0)]
Ken Scott                bool_at_[('Ken Scott', 'PMP', 0)]
Kiourkas                  bool_at_[('Kiourkas', 'PMP', 0)]
Kirby                        bool_at_[('Kirby', 'PMP', 0)]
Lee                            bool_at_[('Lee', 'PMP', 0)]
Leon-Guerrero        bool_at_[('Leon-Guerrero', 'PMP', 0)]
Meek                          bool_at_[('Meek', 'PMP', 0)]
Nelson                      bool_at_[('Nelson', 'PMP', 0)]
O'Neill                    bool_at_[("O'Neill", 'PMP', 0

In [247]:
residents[residents.resident_year == 1].index

Index(['Benito', 'Katsilometes', 'Kiourkas', 'Lee', 'Nelson', 'Oleson',
       'Amanda Scott', 'Sestak', 'Szabo', 'Van Bramer'],
      dtype='object', name='resident_name')

In [248]:
residents = residents[residents.resident_year == 3] # cut down for feasibility

In [249]:
residents

,resident_year
resident_name,
Dewald,3
Fisher,3
Kirby,3
Peters,3
Ruger,3
Ken Scott,3
Solaiman,3
Straus,3
Warr,3


In [250]:
model = cp_model.CpModel()

d = model.NewBoolVarSeries(
    "bool_at_",
    pd.Index((list(it.product(residents.index, rotations.index, weeks.index)))),
).sort_index()  # makes a dummy pd.Series with strings that are labels

# every resident's week x must have exactly 1 rotation
for n, w in it.product(residents.index, weeks.index):
    model.AddExactlyOne(d.loc[pd.IndexSlice[n, :, w]])

# Every rotation can only go one place each week
for r, w in it.product(rotations.index, weeks.index):
    model.AddAtMostOne(d.loc[pd.IndexSlice[:, r, w]])

# apply max capacity to each rotation for each rotation:week index
for r, w in it.product(rotations.index, weeks.index):
    model.Add(sum(d.loc[pd.IndexSlice[:, r, w]]) <= rotations.resident_capacity[r])

# for each rotation, set max weeks to use
for rot_name, rot_tail in rotations.iterrows():
    for res in residents.index:
        if rot_tail["max_wks"] <= MAX_WKS:  # may be
            model.Add(
                sum(d.loc[pd.IndexSlice[res, rot_name, :]]) <= rot_tail["max_wks"]
            )

# # ^^^ set senior requirements
# for rot_name, rot_tail in rotations[(rotations.min_wks > 0) & (rotations.required_level == "Senior")].iterrows():
#     for res in residents[residents.resident_year > 1].index:
#         model.Add(sum(d.loc[pd.IndexSlice[res, rot_name, :]]) >= rot_tail.min_wks)

# # ^^^ set intern requirements
# for rot_name, rot_tail in rotations[(rotations.min_wks > 0) & (rotations.required_level == "Intern")].iterrows():
#     for res in residents[residents.resident_year == 1].index:
#         model.Add(sum(d.loc[pd.IndexSlice[res, rot_name, :]]) >= rot_tail.min_wks)

# for any rotations which require contiguous weeks
rotations_with_contig_reqs = rotations[rotations.min_contig_wks > 1]
for contig_rot, contig_rot_tail in rotations_with_contig_reqs.iterrows():
    print(f"{contig_rot} - {contig_rot_tail.min_contig_wks}")
    for res in residents.index:
        hard_min = contig_rot_tail.min_contig_wks
        works = d.loc[pd.IndexSlice[res,contig_rot,:]]
        for length in range(1,hard_min):
            for start in range(len(works) - length + 1):
                model.AddBoolOr(negated_bounded_span(works,start,length))

print(model.ModelStats())

PMP - 2
HS Senior - 2
HS Intern - 4
SHMC ICU Senior - 4
Prov Rheum - 2
Nights Intern - 4
Nights Senior - 4
CCNW H/O - 2
DH ICU - 4
Prov Pulm - 2
Summit CC - 2
Multicare Pulm - 2
CICU - 2
SHMC ICU Intern - 4
DH ICU Senior - 4
satisfaction model '': (model_fingerprint: 0x6e7214acb5972e1a)
#Variables: 10'070
  - 10'070 Booleans in [0,1]
#kAtMostOne: 1'007 (#literals: 10'070)
#kBoolOr: 15'160 (#literals: 55'680)
#kExactlyOne: 530 (#literals: 10'070)
#kLinearN: 1'197 (#terms: 20'140)


In [251]:
# # TODO for each rotation with a category - could do
# for rot_name, rot_tail in rotations_s_t_categories.iterrows():
#     print(f"{rot_name} -  {rot_tail.category}")
#     for 

In [252]:
# model.Maximize(
#     sum(
#         [
#             d.loc[pd.IndexSlice[res, rot, wk]] * prefs.loc[res, rot]
#             for res in residents.index
#             for rot in rotations.index
#             for wk in weeks.index
#         ]
#     )
# )

In [253]:
solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 4
solver.parameters.log_search_progress = True
solver.log_callback = print
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
    print("optimal")
    # # The status tells us if we were able to compute a solution.
    # for n, w in it.product(residents, weeks):
    #     print(
    #         "{} {}\n{}".format(
    #             n,
    #             w,
    #             solver.Values(d.loc[pd.IndexSlice[n, :, w]])[
    #                 solver.Values(d.loc[pd.IndexSlice[n, :, w]]) != 0
    #             ],
    #         )
    #     )
    print("=====Stats:======")
    print(solver.SolutionInfo())
    print(solver.ResponseStats())
elif status == cp_model.FEASIBLE:
    print("feasible")
    # # The status tells us if we were able to compute a solution.
    # for n, w in it.product(residents, weeks):
    #     print(
    #         "{} {}\n{}".format(
    #             n,
    #             w,
    #             solver.Values(d.loc[pd.IndexSlice[n, :, w]])[
    #                 solver.Values(d.loc[pd.IndexSlice[n, :, w]]) != 0
    #             ],
    #         )
    #     )
    print("=====Stats:======")
    print(solver.SolutionInfo())
    print(solver.ResponseStats())
else:
    print("no solution")


Starting CP-SAT solver v9.12.4544
Parameters: log_search_progress: true num_search_workers: 4

Starting CP-SAT solver v9.12.4544
Parameters: log_search_progress: true num_search_workers: 4


Initial satisfaction model '': (model_fingerprint: 0x6e7214acb5972e1a)
#Variables: 10'070
  - 10'070 Booleans in [0,1]
#kAtMostOne: 1'007 (#literals: 10'070)
#kBoolOr: 15'160 (#literals: 55'680)
#kExactlyOne: 530 (#literals: 10'070)
#kLinearN: 1'197 (#terms: 20'140)
Initial satisfaction model '': (model_fingerprint: 0x6e7214acb5972e1a)
#Variables: 10'070
  - 10'070 Booleans in [0,1]
#kAtMostOne: 1'007 (#literals: 10'070)
#kBoolOr: 15'160 (#literals: 55'680)
#kExactlyOne: 530 (#literals: 10'070)
#kLinearN: 1'197 (#terms: 20'140)

Starting presolve at 0.01s

Starting presolve at 0.01s
  4.58e-03s  0.00e+00d  [DetectDominanceRelations] 
  4.58e-03s  0.00e+00d  [DetectDominanceRelations] 
  7.91e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  7.91e-02s  0.00e+00d  [Pres

In [ ]:
print(solver.Values(d)[solver.Values(d) == 1].sort_index(level=(0, 2)))
# solver.Values(d)[solver.Values(d) == 1].sort_index(level=(0, 2)).to_csv(
#     "D:/Informatics\\opre\\final_tabular.csv"
# )

print(
    solver.Values(d)[solver.Values(d) == 1].sort_index(level=(0, 2)).unstack().to_markdown()
)
# solver.Values(d)[solver.Values(d) == 1].sort_index(
#     level=(0, 2)
# ).unstack().to_csv("D:/Informatics\\opre\\final_grid.csv")

print_full(
    solver.Values(d)[solver.Values(d) == 1]
    .sort_index(level=(0, 2))
    .unstack()
    .reset_index()
    .melt(id_vars=["level_0", "level_1"])
    .query("value == 1")
    .set_index(["level_0", "variable"])
    .sort_index()["level_1"]
    .unstack()
)

In [ ]:
print_full(solver.Values(d)[solver.Values(d) == 1].sort_index(level=(0, 2)).unstack())

In [ ]:
print(
    solver.Values(d)[solver.Values(d) == 1]
    .sort_index(level=(0, 2))
    .unstack()
    .reset_index()
    .melt(id_vars=["level_0", "level_1"])
    .query("value == 1")
    .set_index(["level_0", "variable"])
    .sort_index()["level_1"]
    .unstack()
)

In [171]:
(
    solver.Values(d)[solver.Values(d) == 1]
    .sort_index(level=(0, 2))
    .unstack()
    .reset_index()
    .melt(id_vars=["level_0", "level_1"])
    .query("value == 1")
    .set_index(["level_0", "variable"])
    .sort_index()["level_1"]
    .unstack()
    .to_excel("output_schedule2.xlsx")
)

ValueError: Index contains duplicate entries, cannot reshape